# Résumé automatique de documents financiers
Objectif : transformer rapidement un **rapport financier** en un **résumé clair et chiffré**.

In [ ]:
!pip install openai PyMuPDF python-dotenv

In [ ]:
import os
import fitz  # PyMuPDF
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import pathlib

print("Dossier courant :", pathlib.Path.cwd())

# Charger le .env
env_path = find_dotenv(filename=".env", usecwd=True)
load_dotenv(dotenv_path=env_path, override=True)

api_key = os.getenv("OPENAI_API_KEY")
print(api_key[:4] + "...****")

In [ ]:
chemin_pdf = "./data/teslafinancialreport.pdf"

pdf = fitz.open(chemin_pdf)
texte = ""
for i, page in enumerate(pdf, start=1):
    texte_page = page.get_text()
    texte += f"\n\n=== [PAGE {i}] ===\n" + texte_page.strip()

texte = "\n".join(l.strip() for l in texte.splitlines())

# Limite de longueur
LONGUEUR_MAX = 120_000
if len(texte) > LONGUEUR_MAX:
    texte = texte[:LONGUEUR_MAX]

print("Aperçu du début du texte :\n")
print(texte[:800])

In [ ]:
consignes = (
    "Tu es analyste financier et expert-comptable. On te fournit le texte d'un document financier (rapport annuel, trimestriel, comptes, bilan, annexes).\n\n"
    "Produis une synthèse précise et chiffrée en Markdown selon ce cadre :\n"
    "- Société / Période / Devise : (si repérable)\n"
    "- Résumé exécutif (5–8 lignes)\n"
    "- Chiffres clés (tableau) : | Indicateur | Valeur | Statut | Méthode / Contexte | Période | Page |\n"
    "|---|---:|---|---|---|---:|\n"
    "- Analyse : performance, structure financière, risques, outlook\n"
    "- Références internes : pages/sections à relire\n"
    "Exigences : N'invente aucun chiffre. Si absent : non précisé. 6 à 12 indicateurs maximum. Reste concis : 200–350 mots."
)

In [ ]:
client = OpenAI(api_key=api_key)
modele = "gpt-4o-mini"

reponse = client.responses.create(
    model=modele,
    input=[
        {"role": "system", "content": consignes},
        {"role": "user", "content": texte},
    ],
)

resume = reponse.output_text
print("\n=== RÉSUMÉ FINANCIER GÉNÉRÉ ===\n")
print(resume)
display(Markdown(resume))

## Questions interactives sur le PDF
Posez des questions spécifiques sur le PDF et obtenez des réponses précises avec références de page si possible.

In [ ]:
consignes_questions = (
    "Tu es analyste financier et expert-comptable. On te donne le texte d’un rapport financier. "
    "Réponds uniquement à la question posée, sans inventer de données. "
    "Si la réponse n’est pas claire dans le texte, écris : 'non précisé'. "
    "Quand c’est possible, indique la page d’origine (repère '=== [PAGE X] ===')."
)

while True:
    ma_question = input("\nPosez votre question (ou tapez 'bye' pour quitter) : ")
    if ma_question.strip().lower() == "bye":
        print("👋 Fin de l’interaction. Merci !")
        break
    reponse_question = client.responses.create(
        model=modele,
        input=[
            {"role": "system", "content": consignes_questions},
            {"role": "user", "content": f"Question : {ma_question}\n\nTexte PDF :\n{texte}"},
        ],
    )
    print("\n=== RÉPONSE ===\n")
    print(reponse_question.output_text)
    display(Markdown(reponse_question.output_text))